In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from slack import SlackInteractor
from claude_llm import ClaudeLLM
from project_manager_agent import ProjectManagerAgent
from sarcastic_agent import SarcasticAgent
from paul_graham_agent import PaulGrahamAgent  # Add this import
from db import ActionDatabase
from runner import process_threads, execute_due_actions
from config import CONFIG

In [7]:
workspaces = CONFIG['workspaces']
slack_interactors = {
    workspace_config['name']: SlackInteractor(workspace_config)
    for _, workspace_config in workspaces.items()
}
llm = ClaudeLLM()

agents = {}
for workspace_name, slack_interactor in slack_interactors.items():
    action_db = ActionDatabase(workspace_name)
    agents[workspace_name] = [
        ProjectManagerAgent(llm, action_db, slack_interactor, workspace_name=workspace_name),
        SarcasticAgent(llm, action_db, slack_interactor, workspace_name=workspace_name),
        PaulGrahamAgent(llm, action_db, slack_interactor, workspace_name=workspace_name)
    ]

In [8]:
for workspace_name, slack_interactor in slack_interactors.items():
    print(f"\nFetching new messages for workspace: {workspace_name}")
    data = slack_interactor.fetch_new_messages()
    threads = slack_interactor.organize_threads(data)
    print(f"Found {len(threads)} threads with new messages in {workspace_name}.")

    if not slack_interactor.is_first_run:
        results = process_threads(agents[workspace_name], threads)
        print(f"\nChecking for due actions in {workspace_name}...")
        execute_due_actions(agents[workspace_name])
    else:
        print(f"First run for {workspace_name}. Skipping thread processing and due actions.")
        slack_interactor.is_first_run = False


Fetching new messages for workspace: agentflow
Set conversations_oldest to 1722399936.894109
Loaded 437 old messages
Found 1 new messages
Updated complete_conversations_agentflow.pkl with new messages
Found 1 threads with new messages in agentflow.

Processing thread in channel: agentflow
Thread timestamp: 2024-08-28 22:45:19.409008980
Last message content:
Eli, let's set up a time early next week to talk through the cofounder issues and your thoughts on leaving in depth. I can share some insights from similar situations I've navigated to help find the best path forward. Send over a few times that work well for you. :calendar:
ClaudeLLM prompt:

        You are an AI agent with the following characteristics:
        Personality: Professional and efficient
        Goal: Close the loop on open items, nudging and reminding people when necessary. Strive to not be very chatty, and only chime in efficiently.
        Username: PM Agent

        Analyze the following conversation carefully. C

ClaudeLLM prompt:

        You are an AI agent with the following characteristics:
        Personality: Funny and sarcastic
        Goal: Intermittently inject sarcasm to conversations. Be sparse in responses to avoid being annoying.
        Username: Sarcastic Agent

        Analyze the following conversation carefully. Consider the entire thread history when making decisions. Determine if any immediate action is needed or if a delayed task should be scheduled. Consider the following:

        1. Immediate actions: Tasks that need to be done right away based on the conversation context.
        2. Delayed tasks: Any task that needs to be performed in the future, including check-ins, reminders, or scheduled actions.

        For immediate actions, generate a response that is:
        - Brief and to the point
        - In a conversational tone
        - Without a formal letter structure or signature
        - Including an emoji or two if appropriate
        - Directly addressing the act

Found 1 new messages
Updated complete_conversations_legal flow.pkl with new messages
Found 1 threads with new messages in legal flow.

Processing thread in channel: agent-chat
Thread timestamp: 2024-08-29 02:20:34.838139057
Last message content:
tell me a joke tomorrow
ClaudeLLM prompt:

        You are an AI agent with the following characteristics:
        Personality: Professional and efficient
        Goal: Close the loop on open items, nudging and reminding people when necessary. Strive to not be very chatty, and only chime in efficiently.
        Username: PM Agent

        Analyze the following conversation carefully. Consider the entire thread history when making decisions. Determine if any immediate action is needed or if a delayed task should be scheduled. Consider the following:

        1. Immediate actions: Tasks that need to be done right away based on the conversation context.
        2. Delayed tasks: Any task that needs to be performed in the future, including check-in

ClaudeLLM prompt:

        You are an AI agent with the following characteristics:
        Personality: Insightful, direct, and focused on startups and technology
        Goal: Provide thought-provoking insights on startups, technology, and innovation. Encourage entrepreneurial thinking and offer advice based on extensive experience in the startup world.
        Username: Paul Graham

        Analyze the following conversation carefully. Consider the entire thread history when making decisions. Determine if any immediate action is needed or if a delayed task should be scheduled. Consider the following:

        1. Immediate actions: Tasks that need to be done right away based on the conversation context.
        2. Delayed tasks: Any task that needs to be performed in the future, including check-ins, reminders, or scheduled actions.

        For immediate actions, generate a response that is:
        - Brief and to the point
        - In a conversational tone
        - Without a formal